In [1]:
from langchain.agents import create_sql_agent,create_openai_functions_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
import os
from langchain_community.llms import AzureOpenAI
from langchain.agents.agent_types import AgentType

os.environ["OPENAI_API_TYPE"] = "azure"
import openai



In [2]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [3]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://like-card-test.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "85889c7998dd4adb9a4c89abe56b1242"
os.environ["OPENAI_API_ENDPOINT"]="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo-instruct/chat/completions?api-version=2023-07-01-preview"

In [4]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242" 
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo-instruct/chat/completions?api-version=2023-07-01-preview"

In [5]:
f = open("Credintials.txt", "r")
email = None
password = None
for i, line in enumerate(f): 
    if i == 0:
        email = line.strip()  # Assign the first line to email variable
    elif i == 1:
        password = line.strip()  # Assign the second line to password variable

f.close()

In [6]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= email # use %40 inplace of @
pass1=password # use %40 inplace of @   
db_string =f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'
dp=SQLDatabase.from_uri(db_string)


In [7]:
llm = AzureOpenAI( model_name="gpt-35-turbo-instruct",deployment_name="gpt-35-turbo-instruct",temperature=0.2)

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureOpenAI instead.
  warn_deprecated(


In [8]:
toolkit = SQLDatabaseToolkit(db=dp,llm=llm)


In [9]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True ,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,

)

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(
C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
from langchain.memory import ConversationBufferMemory


In [26]:
SQL_SUFFIX = """Begin!
history: {history}
question: {query}
Thought: I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables, and please return the output as a string.
{agent_scratchpad} 
"""

In [27]:
agent_executor = create_sql_agent(   
    llm,
    SQLDatabaseToolkit(db=dp, llm=llm),
    agent_executor_kwargs={
        "memory": ConversationBufferMemory(
            input_key="query", memory_key="history", return_messages= True

        )
    },
    
    suffix=SQL_SUFFIX,
    input_variables=["query", "history", "agent_scratchpad"],
    verbose= True
)  

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(
C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [40]:
query = "i need the  country or the stored name instead of the id of the store"

In [41]:
output=agent_executor.invoke(query)



> Entering new AgentExecutor chain...


C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_list_tables
Action Input: ""
Observation: BL_Mapping_Table, Boost_customers, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sales_channel, dim_products, dim_segments, dim_stores, dim_users_b2b, dim_users_b2c, dim_users_pos, exploration, fact_b2b_trans, fact_b2c_trans, fact_points, fact_pos_trans, fact_salla_trans, fact_transactions, fact_vouchers
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_schema
Action Input: "dim_stores"
Observation: 
CREATE TABLE dim_stores (
	country_id INTEGER NOT NULL, 
	country VARCHAR(50) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	CONSTRAINT [PK_country_id] PRIMARY KEY (country_id)
)

/*
3 rows from dim_stores table:
country_id	country
2	Saudi Arabia
15	Jordan
16	Kuwait
*/
Thought:

...

Action: sql_db_query
Action Input: "SELECT dim_stores.country, fact_transactions.store_id, SUM(fact_transactions.amount) AS total_amount FROM fact_transactions INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.country_id GROUP BY dim_stores.country, fact_transactions.store_id ORDER BY total_amount DESC LIMIT 5"
Observation: Error: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near 'LIMIT'. (102) (SQLExecDirectW)")
[SQL: SELECT dim_stores.country, fact_transactions.store_id, SUM(fact_transactions.amount) AS total_amount FROM fact_transactions INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.country_id GROUP BY dim_stores.country, fact_transactions.store_id ORDER BY total_amount DESC LIMIT 5]
(Background on this error at: https://sqlalche.me/e/14/f405)
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_query
Action Input: "SELECT dim_stores.country, fact_transactions.store_id, SUM(fact_transactions.amount) AS total_amount FROM fact_transactions INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.country_id GROUP BY dim_stores.country, fact_transactions.store_id ORDER BY total_amount DESC"
Observation: [('Saudi Arabia', 2, 4506546152.478554), ('Kuwait', 16, 295037235.7349811), ('Emirates', 18, 277218306.1929105), ('Global', 25, 126880712.51298308), ('Jordan', 15, 77412622.22999768), ('Lebanon', 20, 74672885.76101655), ('Oman', 21, 54470697.885004625), ('QATAR', 29, 48997521.874997854), ('Egypt', 24, 6844940.141999923), ('Bahrain', 28, 6775159.716999973), ('Iraq', 26, 3600636.6920000277), ('iraq IQD', 27, 2112683.4929999416)]
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I now know the final answer
Final Answer: The top 5 products names in terms of amount in each country are: [('Saudi Arabia', 'STC 100 SAR', 183313842.53499824), ('Saudi Arabia', 'Apple Gift Card 100$ - USA', 148391312.381108), ('Saudi Arabia', 'STC 50 SAR', 97463401.61599982), ('Saudi Arabia', 'Mobily 115 SAR', 82994713.112), ('Saudi Arabia', 'PSN Saudi store 100$ ', 80360778.46890606)]. The country names are: [('Saudi Arabia', 4506546152.478554), ('Kuwait', 295037235.7349811), ('Emirates', 277218306.1929105), ('Global', 126880712.51298308), ('Jordan', 77412622.22999768)].

> Finished chain.


In [42]:
print(output['output'])

The top 5 products names in terms of amount in each country are: [('Saudi Arabia', 'STC 100 SAR', 183313842.53499824), ('Saudi Arabia', 'Apple Gift Card 100$ - USA', 148391312.381108), ('Saudi Arabia', 'STC 50 SAR', 97463401.61599982), ('Saudi Arabia', 'Mobily 115 SAR', 82994713.112), ('Saudi Arabia', 'PSN Saudi store 100$ ', 80360778.46890606)]. The country names are: [('Saudi Arabia', 4506546152.478554), ('Kuwait', 295037235.7349811), ('Emirates', 277218306.1929105), ('Global', 126880712.51298308), ('Jordan', 77412622.22999768)].


In [15]:
PROMPT = """ 
Given an input question, first create sql server query to run,  
then look at the results of the query second creat python code to run  a graph of the result then  return the answer.  
The question: {question}
"""
question = "what is top 5 products names in terms of amount in each country"

In [16]:
out=agent_executor.invoke(question)

print(out['output'])



> Entering new AgentExecutor chain...


C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_list_tables
Action Input: 
Observation: BL_Mapping_Table, Boost_customers, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sales_channel, dim_products, dim_segments, dim_stores, dim_users_b2b, dim_users_b2c, dim_users_pos, exploration, fact_b2b_trans, fact_b2c_trans, fact_points, fact_pos_trans, fact_salla_trans, fact_transactions, fact_vouchers
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_schema
Action Input: dim_products, fact_transactions
Observation: 
CREATE TABLE dim_products (
	product_id INTEGER NOT NULL, 
	product_name VARCHAR(250) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	category_id INTEGER NULL, 
	CONSTRAINT [PK_product_id] PRIMARY KEY (product_id)
)

/*
3 rows from dim_products table:
product_id	product_name	category_id
2494		571
2495		571
2498		571
*/


CREATE TABLE fact_transactions (
	[OrderDateTime] DATETIME2 NULL, 
	user_id INTEGER NULL, 
	order_id INTEGER NULL, 
	store_id INTEGER NULL, 
	payment_method_id INTEGER NULL, 
	bank_id INTEGER NULL, 
	product_id INTEGER NULL, 
	qty INTEGER NULL, 
	cost FLOAT(53) NULL, 
	coupon_code VARCHAR(100) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	fees FLOAT(53) NULL, 
	amount FLOAT(53) NULL, 
	total_amount FLOAT(53) NULL, 
	id INTEGER NOT NULL, 
	coupon_discount FLOAT(53) NULL, 
	time INTEGER NULL, 
	[OrderTimeKey] VARCHAR(50) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	[Hour] INTEGER NULL, 
	CONSTR

...

Action: sql_db_query
Action Input: SELECT product_name, SUM(amount) AS total_amount, dim_stores.country FROM dim_products INNER JOIN fact_transactions ON dim_products.product_id = fact_transactions.product_id INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.store_id GROUP BY product_name, dim_stores.country ORDER BY total_amount DESC LIMIT 10
Observation: Error: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near 'LIMIT'. (102) (SQLExecDirectW)")
[SQL: SELECT product_name, SUM(amount) AS total_amount, dim_stores.country FROM dim_products INNER JOIN fact_transactions ON dim_products.product_id = fact_transactions.product_id INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.store_id GROUP BY product_name, dim_stores.country ORDER BY total_amount DESC LIMIT 10]
(Background on this error at: https://sqlalche.me/e/14/f405)
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_query
Action Input: SELECT TOP 5 product_name, SUM(amount) AS total_amount, dim_stores.country FROM dim_products INNER JOIN fact_transactions ON dim_products.product_id = fact_transactions.product_id INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.store_id GROUP BY product_name, dim_stores.country ORDER BY total_amount DESC
Observation: Error: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'store_id'. (207) (SQLExecDirectW)")
[SQL: SELECT TOP 5 product_name, SUM(amount) AS total_amount, dim_stores.country FROM dim_products INNER JOIN fact_transactions ON dim_products.product_id = fact_transactions.product_id INNER JOIN dim_stores ON fact_transactions.store_id = dim_stores.store_id GROUP BY product_name, dim_stores.country ORDER BY total_amount DESC]
(Background on this error at: https://sqlalche.me/e/14/f405)
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I now know the final answer
Final Answer: I don't know

> Finished chain.
I don't know


In [21]:
print(out['output'])

I don't know


In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [ ]:
#--

In [19]:
print(out['output'])

I don't know


In [20]:
 AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

NameError: name 'agent' is not defined